### Install Packages

In [7]:
pip install keybert

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install nltk

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install HanTa

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install spacy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install treetaggerwrapper

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import Packages

In [12]:
import os
import string
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from keybert import KeyBERT
import pandas as pd

import spacy
from spacy.lang.de import German

import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from HanTa import HanoverTagger as ht
from pprint import pprint

import treetaggerwrapper

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sophiabuehl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Static Variables / Config

In [ ]:
TEXT_FOLDER = './exzellent/'
CLEANED_FOLDER = './cleaned_exzellent/'

### Pre-Process the Excellent Articles

In [ ]:
# https://github.com/wartaal/HanTa
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [13]:
def clean_text(text):
    # Tokenisierung
    words = nltk.word_tokenize(text, language='german')

    # POS-Tagging
    tagged_words = tagger.tag_sent(words)

    # Anpassen der Rückgabestruktur des HanoverTaggers
    pos_tags = [(word, tag) for word, _, tag in tagged_words]

    # Filtern der Wörter nach NN (Nomen), NE (Eigennamen) und ADJ (Adjektive)
    filtered_words = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('ADJ') or pos == 'NE']

    # Zusammenfügen der gefilterten Wörter zu einem bereinigten Text
    cleaned_text = ' '.join(filtered_words)
    
    return cleaned_text

### Clean data and save

In [14]:
# Pfad zum Ordner für die gereinigten Dateien
if not os.path.exists(CLEANED_FOLDER):
    os.makedirs(CLEANED_FOLDER)

# Liste alle TXT-Dateien im Ordner auf
txt_files = [file for file in os.listdir(TEXT_FOLDER) if file.endswith('.txt')]

# Iteriere über alle TXT-Dateien
for txt_file in txt_files:
    # Pfad zur aktuellen TXT-Datei
    file_path = os.path.join(TEXT_FOLDER, txt_file)

    # Lese den Inhalt der TXT-Datei ein
    with open(file_path, 'r') as file:
        text = file.read()

    # Bereinige den Text
    cleaned_text = clean_text(text)

    # Speichere den bereinigten Text in einer neuen Datei im "cleaned_files"-Ordner
    output_file = os.path.splitext(txt_file)[0] + '_cleaned.txt'
    output_path = os.path.join(CLEANED_FOLDER, output_file)
    with open(output_path, 'w') as file:
        file.write(cleaned_text)

### Get keywords and print them

In [15]:
# Erzeuge leere Liste für die Datensätze
data = []

# Durchlaufe alle Textdateien im Ordner
for filename in os.listdir(CLEANED_FOLDER):
    if filename.endswith('.txt'):
        # Lese den Inhalt der Textdatei ein
        txt_path = os.path.join(CLEANED_FOLDER, filename)
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Extrahiere Schlüsselwörter für den Artikelinhalt
        kw_model = KeyBERT()
        keywords = kw_model.extract_keywords(content, keyphrase_ngram_range=(1, 2))

        # Extrahiere den Titel des Artikels aus dem Dateinamen
        title = os.path.splitext(filename)[0]

        # Füge den Datensatz zur Liste hinzu
        data.append({'Titel': title, 'Schlüsselwörter': keywords})

# Erstelle eine Pandas DataFrame aus den Datensätzen
df = pd.DataFrame(data)

# Zeige die Tabelle mit Titel und Schlüsselwörtern an
print(df)

                Titel                                    Schlüsselwörter
0     4291516_cleaned  [(tomus antiochenos, 0.6083), (antiochenos tom...
1       28430_cleaned  [(papstzeremoniells entscheidenden, 0.4779), (...
2       12672_cleaned  [(ministerpräsidenten bismarck, 0.7477), (bism...
3     1200964_cleaned  [(australiens land, 0.6772), (australien land,...
4       11387_cleaned  [(murales diego, 0.6734), (rivera diego, 0.661...
...               ...                                                ...
2788   661153_cleaned  [(türkische archäologe, 0.632), (türkei oberla...
2789  1386366_cleaned  [(finnland heiligenverehrung, 0.5731), (heilig...
2790  2755592_cleaned  [(birmingham band, 0.5529), (musik band, 0.520...
2791    61819_cleaned  [(são paulos, 0.566), (sao paulo, 0.5587), (sã...
2792   444611_cleaned  [(nordfledermaus europäischen, 0.7675), (nordf...

[2793 rows x 2 columns]


### Save into CSV

In [16]:
df.to_csv('./key.csv') 